In [3]:
import pandas as pd
from modeling import feature_engineering, get_best_params,closest, agg_function,agg_diff_fun, save_object, load_object, model_predict, save_csv
import numpy as np
import reverse_geocoder as revgc
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, KFold
from lightgbm import LGBMClassifier



In [2]:
df_train= pd.read_json('train.json')
df_test= pd.read_json('test.json')
metro= pd.read_csv('MetroStanice.csv')
df_train["training"]=1
df_test["training"]=0

df= pd.concat([df_train,df_test],sort=True).reset_index(drop=True)

mapping_features= pd.read_csv('feature_deduplication.csv')
mydict= dict(zip(mapping_features.original_feature, mapping_features.unique_feature))
#features and photos
df["len_features"] = df["features"].apply(len)
df["len_photos"] = df["photos"].apply(len)
df['features']= df['features'].apply(lambda x: [item.lower() for item in x]) 
df['features_txt']= df['features'].apply(lambda x: ' '.join(x))
df['features']=df['features'].apply(lambda x: [mydict[item] if item in mydict else item for item in x ])
#pocet features
#

#timefromepoch
import datetime
#df['tstamp'] =df['created'].apply(lambda x:  datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S').timestamp())
#Datumove
#df["date"]=  df['created'].str[:7]
df['created']=pd.to_datetime(df["created"])
#df['day'] = df['created'].dt.day
#df['days']=(df["created"]-min(df["created"])).apply(lambda x: x.days)
df['dayofweek'] = df['created'].dt.dayofweek
#cylicke
df['hr'] = df['created'].dt.hour
df['mnth'] = df['created'].dt.month
df['nwords_description']= df['description'].apply(lambda x: x.count(' ')+1)
df['price_digits']=df['price'].apply(lambda x: len(str(x)))
df['price'] = np.log(df['price'])

# GEO
coordinates= df[["latitude","longitude"]].values
coordinates_tuple=tuple((x[0],x[1]) for x in coordinates)
adresy=revgc.search(coordinates_tuple)
df=pd.concat([df,pd.DataFrame(adresy)],axis=1)


#label encoding
event_dictionary ={'low' : 1, 'medium' : 2, 'high' : 3} 
df['interest_integer'] =df['interest_level'].map(event_dictionary) 

#interakcie zakladna intuicia
df['bed+bath']=df['bedrooms']+df['bathrooms']
df['price/bed']=df['price']/(df['bedrooms']+1e-16)
df['price/bed+bath']=df['price']/(df['bed+bath']+1e-16)

training='training'
label='interest_integer'

metro=metro.loc[:,['lat','lon']]
metro=metro.to_dict('records')
dfmetro=df.loc[:,['latitude','longitude']]
dfmetro=dfmetro.to_dict('records')
x=[]
for i in dfmetro:
    x.append(closest(metro, i))
df['metro']=x


#drop columns
df=df.drop(columns=['photos','created','listing_id','lat','lon','latitude','longitude'])

Loading formatted geocoded file...


In [3]:
clf=LGBMClassifier(n_jobs=-1,n_estimators=100)
agg_dict={agg_function: ['mean','median','sum','std'],agg_diff_fun: ['mean', 'median']}
training='training'
label='interest_integer'
pca_list=[1,2,3,4,5]
num_features=50
df_out,columns,good_encoders,dp_table,df_tokenizer=feature_engineering(df,training,label,num_features,agg_dict,pca_list,clf=clf)
save_object([df_out,columns,good_encoders,dp_table,df_tokenizer],clf.__class__.__name__+'_thesis')

2020-05-22 23:33:27.281292


100%|██████████| 12/12 [01:48<00:00,  9.02s/it]


after numeric feature selection:  ['bathrooms', 'bedrooms', 'price', 'len_features', 'len_photos', 'hr', 'mnth', 'nwords_description', 'bed+bath', 'price/bed', 'price/bed+bath', 'metro'] 0.631485377550728


  0%|          | 0/7 [00:00<?, ?it/s]

after TOP N features:  ['bathrooms', 'bedrooms', 'price', 'len_features', 'len_photos', 'hr', 'mnth', 'nwords_description', 'bed+bath', 'price/bed', 'price/bed+bath', 'metro', 'laundry', 'elevator', 'hardwood', 'cats allowed', 'dogs allowed', 'concierge', 'no fee', 'fitness center', 'pre war', 'roof', 'swimming pool', 'exclusive', 'common outdoor space', 'simplex', 'fireplace', 'low-rise', 'reduced fee', 'furnished', 'publicoutdoor', 'renovated', 'pool', 'on-site garage', 'light', 'pets'] 0.611096158585503


...

3 <function stemming_tokenizer at 0x7fab08dfd3b0> 1
3 <function stemming_tokenizer at 0x7fab08dfd3b0> 2
3 <function stemming_tokenizer at 0x7fab08dfd3b0> 3
3 <function stemming_tokenizer at 0x7fab08dfd3b0> 4
3 <function stemming_tokenizer at 0x7fab08dfd3b0> 5


/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


3 <modeling.LemmaTokenizer object at 0x7fab099da510> 1
3 <modeling.LemmaTokenizer object at 0x7fab099da510> 2
3 <modeling.LemmaTokenizer object at 0x7fab099da510> 3
3 <modeling.LemmaTokenizer object at 0x7fab099da510> 4
3 <modeling.LemmaTokenizer object at 0x7fab099da510> 5
3 None 1
3 None 2
3 None 3
3 None 4


100%|██████████| 3/3 [43:25<00:00, 868.50s/it]

3 None 5
after text feature tokenization plus PCA:  ['bathrooms', 'bedrooms', 'price', 'len_features', 'len_photos', 'hr', 'mnth', 'nwords_description', 'bed+bath', 'price/bed', 'price/bed+bath', 'metro', 'laundry', 'elevator', 'hardwood', 'cats allowed', 'dogs allowed', 'concierge', 'no fee', 'fitness center', 'pre war', 'roof', 'swimming pool', 'exclusive', 'common outdoor space', 'simplex', 'fireplace', 'low-rise', 'reduced fee', 'furnished', 'publicoutdoor', 'renovated', 'pool', 'on-site garage', 'light', 'pets', 'mean_bathrooms_by_building_id', 'median_bathrooms_by_building_id', 'sum_bathrooms_by_building_id', 'std_bathrooms_by_building_id', 'bathrooms-mean_bathrooms_by_building_id', 'bathrooms-median_bathrooms_by_building_id', 'mean_bedrooms_by_building_id', 'median_bedrooms_by_building_id', 'sum_bedrooms_by_building_id', 'std_bedrooms_by_building_id', 'bedrooms-mean_bedrooms_by_building_id', 'bedrooms-median_bedrooms_by_building_id', 'mean_price_by_building_id', 'median_price_by

In [4]:
parameters=pd.read_csv('LGBMClassifier_grid_results.csv')
best_params=get_best_params(parameters,'score')
best_params

{'n_estimators': 972,
 'learning_rate': 0.01848416124154692,
 'lambda_l1': 1.1948456836604695,
 'lambda_l2': 8.364115989935963e-07,
 'num_leaves': 34,
 'feature_fraction': 0.7671799905376775,
 'bagging_fraction': 0.7281188949960176,
 'bagging_freq': 2,
 'min_child_samples': 88}

In [ ]:
parameters=pd.read_csv('LGBMClassifier_grid_results.csv')
best_params=get_best_params(parameters,'score')
best_params
clf=LGBMClassifier(n_jobs=-1,**best_params)


In [3]:
parameters=pd.read_csv('LGBMClassifier_grid_results.csv')
get_best_params(parameters,'score')

{'n_estimators': 737,
 'learning_rate': 0.016592015076447413,
 'lambda_l1': 0.6697436211113311,
 'lambda_l2': 0.0043913703402733316,
 'num_leaves': 59,
 'feature_fraction': 0.7643364611332195,
 'bagging_fraction': 0.6126062676849265,
 'bagging_freq': 5,
 'min_child_samples': 90}

In [2]:
parameters=pd.read_csv('LGBMClassifier_grid_results.csv')
get_best_params(parameters,'score')
save_object([df_out,columns,clf,good_encoders],clf.__class__.__name__+'best_grid_params')

NameError: name 'df_out' is not defined